In [ ]:
from kami_reports.database import get_dataframe_from_sql_query
from datetime import datetime as dt, timedelta as td
from kami_reports.constant import BEEXP_SCRIPT, ORDER_ITEM_DATE_COLS
from messages import send_messages_by_group
first_update = dt.now() - td(days=7)
current_update = first_update
last_update = first_update
def get_beexp_since(ordered_date):
    oders_df = get_dataframe_from_sql_query(
    sql_script=BEEXP_SCRIPT,
    date_cols=ORDER_ITEM_DATE_COLS
    )
    mask = (oders_df['dt_implant'] >= ordered_date)
    return oders_df.loc[mask]

beexp_df = get_beexp_since(current_update)
products = [f"Quantidade de Ingressos: [dado] Nome do Vendedor: [dado] Tipo de Ingresso: [dado] Valor faturado: [dado] Forma de pagamento: [dado] Código do cliente: [dado] Data da venda: [dado] E-mail do representante: [dado] O respectivo vendedor já recebeu o link de cadastro do participante." for cod in ]
send_messages_by_group(
        message_body=products, group='comercial'
)

In [ ]:
script = 'database/ase/ase/ase/ase/ase/in/beexp_orders.sql'
from kami_reports.database import get_dataframe_from_sql
beexp_df = get_dataframe_from_sql(script)     
beexp_filtered_df = beexp_df[['qtd', 'nome_colaborador', 'desc_comercial', 'preco_total', 'cond_pgto', 'cod_cliente', 'nome_cliente', 'dt_implant', 'email_colaborador']]

In [ ]:
from mkt_automation import get_beexp_since, get_beexp_manager_message
from datetime import datetime as dt, timedelta as td
from messages import send_messages_by_group
first_update = dt.now() - td(days=7)
current_update = first_update
last_update = first_update

beexp_df = get_beexp_since(current_update)
beexp_df = beexp_df[['qtd', 'nome_colaborador', 'desc_comercial', 'preco_total', 'cond_pgto', 'cod_cliente', 'nome_cliente', 'dt_implant', 'email_colaborador']]
for row in beexp_df.itertuples():
  beexp_message = get_beexp_manager_message(row)
  send_messages_by_group(message_body=beexp_message, group='beexp_manager')

In [ ]:
import polars as pl
sellers_df = pl.read_csv('database/in/vendedores.csv', separator=';')
sellers_df = sellers_df.select(
    pl.col('cod_colaborador'),
    pl.col('nome_colaborador'),
    pl.col('ddd_celular').cast(pl.Utf8).alias('ddd_celular'),
    pl.col('celular').cast(pl.Utf8).alias('celular'),
    pl.col('ddd_1').cast(pl.Utf8).alias('ddd_1'),
    pl.col('telefone_1').cast(pl.Utf8).alias('telefone_1'),
    pl.col('email')
)

In [ ]:
sellers_contacts = sellers_df.with_columns([
    pl.when((
        pl.col('ddd_celular').is_not_null()) & (pl.col('celular').is_not_null()))
    .then(pl.map(['ddd_celular', 'celular'], lambda s: '+55'+s[0]+s[1]))
    .otherwise(None)
    .alias('cellphone')    
])

sellers_contacts = sellers_contacts.with_columns([
    pl.when((
        pl.col('ddd_1').is_not_null()) & (pl.col('telefone_1').is_not_null()))
    .then(pl.map(['ddd_1', 'telefone_1'], lambda s: '+55'+s[0]+s[1]))
    .otherwise(None)
    .alias('phonenumber')
])

sellers_contacts = sellers_contacts.with_columns([
    pl.when(pl.col('cellphone').is_not_null())
    .then(pl.col('cellphone'))    
    .otherwise(None)
    .alias('phone')
])

sellers_contacts = sellers_contacts.with_columns([
    pl.when(pl.col('phone').is_not_null())
    .then(pl.col('phone'))    
    .otherwise(pl.col('phonenumber'))
    .alias('phone')
])

sellers_contacts = sellers_contacts.drop_nulls(['phone']).select(pl.col(['cod_colaborador', 'nome_colaborador','phone','email']))

sellers_contacts = sellers_contacts.with_columns([
    pl.when(pl.col('cod_colaborador').is_not_null())
    .then(pl.lit("['beexp_seller']"))    
    .otherwise(pl.lit("['beexp_seller']"))
    .alias('groups')
])

sellers_contacts.columns = ['uno_id', 'name', 'phone', 'email', 'groups']

In [ ]:
sellers_contacts.write_json('messages/sellers_contacts.json', row_oriented=True)

In [ ]:
sellers_contacts_bot_conversa = sellers_contacts.select(pl.col(['phone', 'name', 'uno_id']))
sellers_contacts_bot_conversa = sellers_contacts_bot_conversa.with_columns([
    pl.when(pl.col('uno_id').is_not_null())
    .then(pl.col('uno_id').cast(pl.Utf8).apply(lambda x: f"__uno__.sg_colaborador.{x}"))
    .otherwise(pl.lit(None))
    .alias('etiquetas')
]).select(pl.col(['phone', 'name', 'etiquetas']))
sellers_contacts_bot_conversa.columns = ['telefone', 'nome', 'etiquetas']

In [ ]:
sellers_contacts_bot_conversa.write_excel('database/out/vendedores_uno_botconversa.xlsx')

In [ ]:
script = 'database/in/sales_teams.sql'
from kami_reports.database import get_dataframe_from_sql
contact_sellers = get_dataframe_from_sql(script)
contact_sellers['groups'] = 'beexp_seller'

In [ ]:
from dataclasses import dataclass
from typing import List

@dataclass
class Contact:

    id: str
    name: str
    email: str
    phone: str
    groups: List[str]
contacts = contact_sellers.apply(lambda row: Contact(**row), axis=1)

In [ ]:
for contact in contacts:
    contact.groups = [contact.groups]

In [ ]:
from messages import get_contact_by_id, get_sellers_contacts_from_database, generate_message_by_template, send_message_by_messenger, send_message_by_all_messengers, send_message_by_group, send_message_by_seller_id

sellers_contacts = get_sellers_contacts_from_database()

In [ ]:
from messages import get_contacts_from_json, send_message_by_group
contacts = get_contacts_from_json('messages/contacts.json')
send_message_by_group(
    'beexp_manager',
    'test',
    {'subject': 'Vendas BEEXP', 'beexp_order': 'teste: t, data: d, tudo ok'},
    contacts
)

In [ ]:
script = 'database/in/sales_teams.sql'
from kami_reports.database import get_dataframe_from_sql
sales_teams = get_dataframe_from_sql(script)
sales_teams

In [ ]:
from dataframe import get_sales_lines_df
sales_lines_df = get_sales_lines_df()
sales_lines_df.columns

In [ ]:
group_columns = ['ano', 'mes', 'empresa_pedido', 'empresa_nota_fiscal', 'cod_cliente',
                  'cod_pedido', 'nr_ped_compra_cli', 'cod_situacao', 'desc_situacao',
                  'nop', 'cfop', 'cod_cond_pagto', 'cod_forma_pagto', 'forma_pgto',
                  'cod_grupo_produto', 'desc_grupo_produto', 'cod_grupo_pai', 'desc_grupo_pai',
                  'cod_marca', 'desc_marca']
sum_columns = ['desconto_pedido', 'valor_nota', 'total_bruto']

sales_orders_df = sales_lines_df.groupby(group_columns)[sum_columns].sum().reset_index()

In [ ]:
import pandas as pd
def create_cod_desc_dataframes(df):
    dataframes_list = []
    for cod_col in df.columns:
        if cod_col.startswith('cod_'):
            col_pair_name = cod_col.split('cod_')[1]
            desc_col = f'desc_{col_pair_name}'            
            if desc_col in df.columns:
                dataframe = df[[cod_col, desc_col]].drop_duplicates()
                df_name = f'{col_pair_name}_df'
                dataframes_list.append({
                  'df_name':df_name,
                  'dataframe':dataframe
                })

    return dataframes_list

In [ ]:
dfs = create_cod_desc_dataframes(sales_orders_df)

In [ ]:
dfs[0]['dataframe']

In [ ]:
import pandas as pd
from typing import List, Union
def get_grouped_sales_orders(sales_lines_df: pd.DataFrame, group_cols: List[str], sum_cols: List[str]) -> pd.DataFrame:
    sales_orders_df = pd.DataFrame()
    """
    Group and aggregate sales data based on specified columns.

    Args:
        sales_lines_df (pd.DataFrame): The original DataFrame with sales data.
        group_cols (List[str]): List of columns to be used for grouping.
        sum_cols (List[str]): List of columns to be summed.

    Returns:
        pd.DataFrame: The resulting DataFrame with aggregated sales orders.
    """
    try:
        sales_orders_df = sales_lines_df.groupby(group_cols)[sum_cols].sum().reset_index()    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    return sales_orders_df

In [ ]:
group_columns = ['ano', 'mes', 'empresa_pedido', 'empresa_nota_fiscal', 'cod_cliente',
                  'cod_pedido', 'nr_ped_compra_cli', 'cod_situacao', 'desc_situacao',
                  'nop', 'cfop', 'cod_cond_pagto', 'cod_forma_pagto', 'forma_pgto',
                  'cod_grupo_produto', 'desc_grupo_produto', 'cod_grupo_pai', 'desc_grupo_pai',
                  'cod_marca', 'desc_marca']
sum_columns = ['desconto_pedido', 'valor_nota', 'total_bruto']
resulting_df = get_grouped_sales_orders(sales_lines_df, group_columns, sum_columns)
resulting_df

In [ ]:
def add_seller_info_to_orders(sales_orders_df: pd.DataFrame, sellers_details_df: pd.DataFrame) -> pd.DataFrame:
    updated_sales_orders_df = pd.DataFrame()
    """
    Add 'cod_colaborador', 'nome_colaborador' columns to sales orders DataFrame based on sellers_customers DataFrame.

    Args:
        sales_orders_df (pd.DataFrame): Sales orders DataFrame.
        sellers_details_df (pd.DataFrame): DataFrame containing seller information.

    Returns:
        pd.DataFrame: Updated sales orders DataFrame with seller information, or None if an error occurs.
    """
    try:        
        updated_sales_orders_df = pd.merge(sales_orders_df, sellers_details_df, on='cod_cliente', how='left')
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    return updated_sales_orders_df 

In [ ]:
import pandas as pd
from kami_reports.database import get_qy_default_seller, get_qy_participant_seller, get_qy_contact_sellers
def get_sellers_details() -> pd.DataFrame:
    sellers_details_df = pd.DataFrame()
    try:
        sellers_details_df = get_qy_contact_sellers()[['id', 'name']].rename(columns={'id': 'cod_colaborador', 'name': 'nome_colaborador'})
        sellers_customers_df = pd.concat([get_qy_default_seller(), get_qy_participant_seller()], ignore_index=True)         
        sellers_details_df = sellers_customers_df.merge(
            sellers_details_df,
            on=['cod_colaborador'],            
            how='left',            
        )
    except Exception as e:
        print('An unknow error occurred:', e)
    return sellers_details_df 

In [ ]:
df = get_sellers_details()
customer_seller_count = df.groupby('cod_cliente')['cod_colaborador'].nunique().reset_index()
customer_seller_count = customer_seller_count.rename(columns={'cod_colaborador': 'unique_seller_count'})

In [ ]:
customer_seller_count
customers_with_multiple_sellers = customer_seller_count[customer_seller_count['unique_seller_count'] > 1]
result_df = pd.merge(df, customers_with_multiple_sellers, on='cod_cliente', how='inner')
result_df


In [ ]:
from kami_reports.database import get_qy_sales_teams
sales_teams_df = get_qy_sales_teams()

In [ ]:
from report_bot import delete_files_from, create_gdrive_folders, deliver_master_reports, send_message_by_group, get_contacts_from_json
from dataframe import build_sales_orders_df, build_master_df, get_sales_lines_df, calculate_master_df, add_sales_teams_on_sellers_df, add_sellers_on_sales_df
import pandas as pd
delete_files_from('database/out')
current_folders_id = create_gdrive_folders()
contacts = get_contacts_from_json('messages/contacts.json')
sales_lines_df = get_sales_lines_df()
sales_orders_df = build_sales_orders_df(sales_lines_df)     
master_sales_teams_df = pd.DataFrame()
master_sellers_df = pd.DataFrame()
master_df = pd.DataFrame()
master_df = calculate_master_df(sales_orders_df)
master_sellers_df = add_sellers_on_sales_df(master_df)
master_sales_teams_df = add_sales_teams_on_sellers_df(master_sellers_df)

In [ ]:
deliver_master_reports(master_sales_teams_df, current_folders_id)

In [ ]:
send_message_by_group(
    template_name='comercial',
    group='comercial',
    message_dict={
        'subject': 'Faturamento Geral Diário',
        'gdrive_folder_id': current_folders_id['comercial'],
    },
    contacts=contacts,
)

In [6]:
import os
current_dir =  os.path.abspath(os.path.dirname(__file__))
parent_dir = os.path.abspath(current_dir + "/../")

NameError: name '__file__' is not defined